# Plot cleaned station coverage
Create figures of station coverage for cleaned stations on a map of WECC. Also enables figure generation by individual variables. 

In [ ]:
import boto3
import pandas as pd
import matplotlib.pylab as plt
from io import BytesIO
import geopandas as gpd
import contextily as cx
import numpy as np

In [ ]:
# All the variables
vars_all = [
    "tas",
    "tdps",
    "tdps_derived",
    "ps",
    "psl",
    "ps_altimeter",
    "pr",
    "pr_5min",
    "pr_1h",
    "pr_24h",
    "pr_localmid",
    "hurs",
    "sfcwind",
    "sfcwind_dir",
    "rsds",
]

In [ ]:
# Read in our specific pre-set per network colormap
# combination of tab20c_r + tab20b colormaps
color_dict = {}
with open("../data/network_colors.txt") as f:
    for line in f:
        (key, val) = line.split()
        color_dict[key] = str("#") + str(val)


In [ ]:
def var_fullname(var):
    """
    Returns a descriptive full name for a variable, formatted for plot titles.

    Parameters
    ----------
    var : str
        Short variable name (e.g., 'tas', 'pr', 'hurs').

    Returns
    -------
    str
        Descriptive title for the variable including units or category.
    """
    if var == "tas":
        var_title = "Air temperature ({})".format(var)
    elif "tdps" in var:
        var_title = "Dewpoint temperature ({})".format(var)
    elif var == "hurs":
        var_title = "Relative humidity ({})".format(var)
    elif var == "rsds":
        var_title = "Radiation ({})".format(var)
    elif var == "sfcwind":
        var_title = "Surface wind speed ({})".format(var)
    elif var == "sfcwind_dir":
        var_title = "Surface wind direction ({})".format(var)
    elif "pr" in var:
        var_title = "Precipitation ({})".format(var)
    elif "ps" in var and "td" not in var:
        var_title = "Air pressure ({})".format(var)
    else:
        var_title = var

    return var_title

In [ ]:
def gdf_setup(var):
    """
    Prepares a GeoDataFrame of stations clipped to the WECC boundary and filtered by data availability.

    Parameters
    ----------
    var : str
        Variable name to subset stations by (e.g., 'tmin', 'tmax', 'prcp').

    Returns
    -------
    GeoDataFrame
        A GeoDataFrame of stations within the WECC boundary that have observations for the given variable.
    """

    # Read in all stations
    df_all = pd.read_csv("s3://wecc-historical-wx/2_clean_wx/temp_clean_all_station_list.csv")

    # Make a geodataframe
    gdf = gpd.GeoDataFrame(
        df_all, geometry=gpd.points_from_xy(df_all.longitude, df_all.latitude)
    )
    gdf.set_crs(epsg=4326, inplace=True)  # Set CRS

    # Project data to match base tiles.
    gdf_wm = gdf.to_crs(epsg=3857)  # Web mercator

    # Read in geometry of WECC
    mar = gpd.read_file(
        "s3://wecc-historical-wx/0_maps/WECC_Informational_MarineCoastal_Boundary_marine.shp"
    )
    ter = gpd.read_file(
        "s3://wecc-historical-wx/0_maps/WECC_Informational_MarineCoastal_Boundary_land.shp"
    )
    wecc = gpd.GeoDataFrame(pd.concat([mar, ter]))

    # Use to clip stations
    wecc = wecc.to_crs(epsg=3857)
    gdf_wecc = gdf_wm.clip(wecc)
    gdf_wecc = gdf_wecc.sort_values(["network"])

    # Setting color
    gdf_wecc["Color"] = gdf_wecc["network"].map(color_dict)

    # Subsetting based on variable
    new_gdf = gdf_wecc.loc[gdf_wecc[str(var) + "_nobs"] > 0]

    return new_gdf

In [ ]:
def single_var_map(var, save_to_aws=False, save_local=False):
    """
    Generates and optionally uploads a map of weather stations for a given variable.

    Parameters
    ----------
    var : str
        Short variable name (e.g., 'tmin', 'tmax', 'prcp').
    save_to_aws : bool, optional
        If True, the figure is saved and uploaded to the AWS S3 bucket.

    Returns
    -------
    None
        Displays the map and optionally uploads it to S3.
    """

    # Set name of saved figure 
    figname = f"clean_station_{var}.png"

    # Grab gdf per variable
    gdf = gdf_setup(var)

    # Figure global settings
    fig, ax = plt.subplots(figsize=(9, 9))
    a = 1  # alpha
    ms = 2  # markersize

    # Enforce WECC boundary, all plots regardless of variable selection
    ylim = [3.5e6, 8.5e6]  # lat
    xlim = [-1.53e7, -1.13e7]  # lon
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)

    # Plot by network with correct legend color
    for ctype, data in gdf.groupby("network"):
        color = color_dict[ctype]
        data.plot(color=color, markersize=ms, alpha=a, ax=ax, label=ctype)

    # Add basemap 
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

    # Add a legend 
    l = ax.legend(loc="lower left", prop={"size": 8}, frameon=True)

    # set title
    vartitle = var_fullname(var)
    ax.set_title(vartitle, fontsize=10)
    ax.set_axis_off()

    # save to AWS
    if save_to_aws: 

        s3 = boto3.resource("s3")
        bucket_name = "wecc-historical-wx"

        img_data = BytesIO()
        plt.savefig(img_data, format="png", bbox_inches="tight")
        img_data.seek(0)

        bucket = s3.Bucket(bucket_name)
        bucket.put_object(
            Body=img_data,
            ContentType="image/png",
            Key=f"2_clean_wx/{figname}"
        )

    if save_local: 
        plt.savefig(f"../figures/{figname}", dpi=300)
    
    return None

In [ ]:
single_var_map(var="tas", save_to_aws=False, save_local=True)

In [ ]:
# generate single maps for all variables
for var in vars_all:
    single_var_map(var=var)

In [ ]:
def plot_combined_station_vars(
    var_list, 
    ncols=3, 
    legend_fontsize=8, 
    save_to_aws=False, 
    save_local=False
):
    """
    Plots a combined figure with weather station maps for multiple variables and optionally saves the figure.

    Parameters
    ----------
    var_list : list of str
        List of variable names to plot (e.g., ['tas', 'pr', 'hurs']).
    ncols : int, optional
        Number of columns in the subplot grid. Default is 3.
    legend_fontsize : int, optional
        Font size for legend labels. Default is 8.
    save_to_aws : bool, optional
        If True, saves the figure and uploads to AWS S3.
    save_local : bool, optional
        If True, saves the figure locally in ../figures/.

    Returns
    -------
    None
    """

    # Set figure name
    if len(var_list) > 10:
        figname = "clean_station_allvars.png"
    else:
        figname = "clean_station_" + "_".join(var_list) + ".png"

    # Plotting configuration
    a = 1      # alpha
    ms = 2     # marker size
    n = 1      # subplot index
    nrows = int(np.ceil(len(var_list) / ncols))

    # Initialize figure
    fig = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8 * ncols, 6 * nrows))

    for var_to_plot in var_list:
        # Create subplot
        ax = plt.subplot(nrows, ncols, n)

        # Set map bounds (Web Mercator)
        ax.set_ylim([3.5e6, 8.5e6])           # latitude bounds
        ax.set_xlim([-1.53e7, -1.13e7])       # longitude bounds

        # Get clipped station GeoDataFrame for this variable
        gdf = gdf_setup(var_to_plot)

        # Plot by network
        for ctype, data in gdf.groupby("network"):
            color = color_dict[ctype]
            data.plot(color=color, markersize=ms, alpha=a, ax=ax, label=ctype)

        # Add base map tiles
        cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

        # Set title
        ax.set_title(var_fullname(var_to_plot), fontsize=12)
        ax.set_axis_off()

        # Add legend
        ax.legend(loc="lower left", prop={"size": legend_fontsize}, frameon=True)

        # Advance subplot index
        n += 1

    # Adjust spacing
    plt.tight_layout()

    # Save to AWS S3
    if save_to_aws:
        # Save figure to in-memory buffer
        img_data = BytesIO()
        plt.savefig(img_data, format="png", bbox_inches="tight", dpi=300)
        img_data.seek(0)  # rewind to beginning after writing

        s3 = boto3.resource("s3")
        bucket = s3.Bucket("wecc-historical-wx")
        bucket.put_object(
            Body=img_data,
            ContentType="image/png",
            Key=f"2_clean_wx/{figname}"
        )

    # Save to local directory 
    if save_local:
        plt.savefig(f"../figures/{figname}", dpi=300, bbox_inches="tight")

    return None


In [ ]:
plot_combined_station_vars(
    vars_all, 
    save_to_aws=False, 
    save_local=True
)

In [ ]:
plot_combined_station_vars(
    ["tas", "tdps", "hurs"],
    save_to_aws=False,
    save_local=True
)